In [3]:
import monai
import pydicom
from monai.config import print_config
from monai.data import Dataset, DataLoader
from monai.metrics import ROCAUCMetric
from monai.networks.nets import DenseNet121
from monai.networks.nets import DenseNet
from monai.utils import first
from monai.transforms import (
    Activations,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,
    AsChannelLastd,
    SqueezeDimd,
    AddChanneld,
    NormalizeIntensityd,
    Resized,
    MapTransform
)
#from monai.handlers.utils import from_engine
from monai.config import print_config
import torch
import os
from natsort import natsorted
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import lightning as pl

print_config()



IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


MONAI version: 1.2.0
Numpy version: 1.24.4
Pytorch version: 2.0.1+cu117
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: c33f1ba588ee00229a309000e888f9817b4f1934
MONAI __file__: /home/dspaanderman/Documents/GitProjects/HabbieDicomSort/venv/lib/python3.8/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: 5.3.0
Nibabel version: 5.1.0
scikit-image version: 0.21.0
Pillow version: 10.0.0
Tensorboard version: 2.14.0
gdown version: 4.7.1
TorchVision version: 0.15.2+cu117
tqdm version: 4.66.1
lmdb version: 1.4.1
psutil version: 5.9.5
pandas version: 2.0.3
einops version: 0.6.1
transformers version: 4.21.3
mlflow version: 2.6.0
pynrrd version: 1.0.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [4]:

# Parent directory where "train" and "test" folders are located
parent_directory = '//mnt/c/Users/ahmad/OneDrive - Delft University of Technology/studi/TUD/Internship/Clean_data/'

parent_directory = 'data/'

# Function to extract label from the folder path
def extract_label(folder_path):
    # Split the folder path and take the last two elements as the label
    components = folder_path.split(os.path.sep)
    label = os.path.join(components[-1])
    return label

# Create a list of dictionaries with keys "path" and "label"
train_list, test_list = [], []

# Iterate through "train" and "test" folder
for split in ["Train Sample", "Test Sample"]:
    split_folder = os.path.join(parent_directory, split)
    for root, _, files in os.walk(split_folder):
        if files:
            dicom_files_path = natsorted([os.path.join(root, file) for file in files if file.lower().endswith('.dcm')])
            # Now instead of finding the label for all these files, lets do it just for the first index, Note I am not using the extract_label function you made
            label = os.path.basename(os.path.normpath(root))
            if  label == "T1":
                class_label = 0
            elif label == "T2":
                class_label = 1
            else :
                class_label = 2
            if split == "Train Sample":
                train_list.append({"image": dicom_files_path, "label": class_label})
            elif split == "Test Sample":
                test_list.append({"image": dicom_files_path, "label": class_label})      

In [5]:
# Define a custom Dataset class to load DICOM files and extract data
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        super().__init__(data, transform)

    def __getitem__(self, index):
        dicom_files_path = self.data[index]["image"]
        label = self.data[index]["label"]
        
        # Extract dicom info from first entry
        dicom_data = pydicom.dcmread(dicom_files_path[0])
        
        # DICOM header attributes
        patient_ID = str(dicom_data.PatientName)
        if hasattr(dicom_data, 'RepetitionTime'):
            repetition_time = int(float(dicom_data.RepetitionTime))
            
        if hasattr(dicom_data, 'EchoTime'):
            echo_time = int(float(dicom_data.EchoTime))
        
        if hasattr(dicom_data, 'MagneticFieldStrength'):
            magnetic_strength = str(dicom_data.MagneticFieldStrength)

        return {"image": dicom_files_path, "label": label, "patient_name": patient_ID, "metadata": {"repetition_time": repetition_time, "echo_time": echo_time, "magnetic_strength": magnetic_strength, "number_slices": len(dicom_files_path)}}

# Create a MONAI dataset
train = CustomDataset(data=train_list)
test = CustomDataset(data=test_list)

In [6]:
def calculate_slice_median(data):
    median = int(np.median([item['metadata']['number_slices'] for item in data]))
    if (median%2 != 0):
        median += 1
    return median

train_slice_median, test_slice_median = calculate_slice_median(train), calculate_slice_median(test)
print(train_slice_median,test_slice_median)

28 40


In [7]:
train[0]

{'image': ['data/Train Sample/LipoRadiomics-028/LipoRadiomics-028/T1/00000.dcm',
  'data/Train Sample/LipoRadiomics-028/LipoRadiomics-028/T1/00001.dcm',
  'data/Train Sample/LipoRadiomics-028/LipoRadiomics-028/T1/00002.dcm',
  'data/Train Sample/LipoRadiomics-028/LipoRadiomics-028/T1/00003.dcm',
  'data/Train Sample/LipoRadiomics-028/LipoRadiomics-028/T1/00004.dcm',
  'data/Train Sample/LipoRadiomics-028/LipoRadiomics-028/T1/00005.dcm',
  'data/Train Sample/LipoRadiomics-028/LipoRadiomics-028/T1/00006.dcm',
  'data/Train Sample/LipoRadiomics-028/LipoRadiomics-028/T1/00007.dcm',
  'data/Train Sample/LipoRadiomics-028/LipoRadiomics-028/T1/00008.dcm',
  'data/Train Sample/LipoRadiomics-028/LipoRadiomics-028/T1/00009.dcm',
  'data/Train Sample/LipoRadiomics-028/LipoRadiomics-028/T1/00010.dcm',
  'data/Train Sample/LipoRadiomics-028/LipoRadiomics-028/T1/00011.dcm',
  'data/Train Sample/LipoRadiomics-028/LipoRadiomics-028/T1/00012.dcm',
  'data/Train Sample/LipoRadiomics-028/LipoRadiomics-02

In [90]:
# I Fixed the loading of the image, now the dicoms are read correctly into the Z axis.

train_transform = Compose(
    [
        LoadImaged(keys=["image"], image_only=True, reader="ITKReader", channel_dim=-1, prune_meta_pattern="|"),
        EnsureChannelFirstd(keys="image", channel_dim="no_channel"),
        NormalizeIntensityd(keys="image", nonzero=False), 
        Resized(keys="image", spatial_size=(256,256,64), size_mode='all', mode='trilinear', align_corners=True), 
    ]
)
test_transform = Compose(
    [
        LoadImaged(keys=["image"], image_only=True, reader="ITKReader", channel_dim=-1, prune_meta_pattern="|"),
        EnsureChannelFirstd(keys="image", channel_dim="no_channel"),
        NormalizeIntensityd(keys="image", nonzero=False), 
        Resized(keys="image", spatial_size=(256,256,64), size_mode='all', mode='trilinear', align_corners=True), 
    ]
)

In [97]:
train_dataset = monai.data.Dataset(train, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=1)

test_dataset = monai.data.Dataset(test, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=2)

In [98]:
check_data = first(test_loader)
image = (check_data["image"])
print(f"image shape: {image.shape}")

image shape: torch.Size([2, 1, 256, 256, 64])


In [ ]:
from monai.visualize import matshow3d
check_data = first(test_loader)
image = (check_data["image"])
print(f"image shape: {image.shape}")

matshow3d(
    volume=check_data["image"],
    fig=None,
    title="input image",
    figsize=(100, 100),
    frame_dim=-1,
    show=True,
    cmap="gray",
)

image shape: torch.Size([1, 256, 256, 18])


(<Figure size 10000x10000 with 1 Axes>,
 array([[       nan,        nan,        nan, ...,        nan,        nan,
                nan],
        [       nan, -0.7102185, -0.7102185, ..., -0.7102185, -0.7102185,
                nan],
        [       nan, -0.7102185, -0.7102185, ..., -0.7102185, -0.7102185,
                nan],
        ...,
        [       nan, -0.7102185, -0.7102185, ...,        nan,        nan,
                nan],
        [       nan, -0.7102185, -0.7102185, ...,        nan,        nan,
                nan],
        [       nan,        nan,        nan, ...,        nan,        nan,
                nan]], dtype=float32))

In [21]:
train_dataset

In [ ]:
class DeepDicomSort(pl.LightningModule):
    def __init__(self, num_classes):
        super().__init__()
        # Here we define the model used for classification, e.g. DenseNet121
        self.model = DenseNet(spatial_dims=3, in_channels=1, out_channels=num_classes)

        # Here we can define the loss function we want to use
        self.loss_function = torch.nn.CrossEntropyLoss()

        # Define activation function for prediction
        self.activation_function = Activations(softmax=True) # Softmax for multiple class and Sigmoid for binary (2) classes

        # Set a Metric for validation set
        self.metric = ROCAUCMetric()
        self.best_val_AUC = 0
        self.best_val_epoch = 0
        
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        # This is the training step, done at each epoch, we load the image and label
        x, y = batch["image"], batch["label"]
        # We run the image forward through the model (DenseNet121)
        y_hat = (self.forward(x))
        print(f"y_pred: , {y_hat[0].shape}")
        # We calculate the loss based on the output of the model and the true labels
        loss = self.loss_function(y_hat, y)
        # We report the loss function
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        # We do the first step similar to training
        x, y = batch["image"], batch["label"]
        y_hat = self.forward(x)
        print(f"y_pred: , {y_hat[0].shape}")
        loss = self.loss_function(y_hat, y)

        # Additionally we can calculate the metric (AUC on the ROC) for each step of the validation
        y_hat = self.activation_function(y_hat)
        self.metric(y_pred=y_hat, y=y)
        self.log('val_loss', loss)
        return loss

    def on_validation_epoch_end(self):
        # Now when a complete validation epoch has done we can report the AUC on the whole set
        val_AUC = self.metric.aggregate()
        self.metric.reset()
        if val_AUC > self.best_val_AUC:
            self.best_val_AUC = val_AUC
            self.best_val_epoch = self.current_epoch
        print(
            f"current epoch: {self.current_epoch} "
            f"current AUC: {val_AUC:.4f}"
            f"\nbest val AUC: {self.best_val_AUC:.4f} "
            f"at epoch: {self.best_val_epoch}"
        )
        self.log("current epoch", self.current_epoch, on_epoch=True)
        self.log("current AUC", val_AUC, on_epoch=True)
        self.log("best AUC", self.best_val_AUC, on_epoch=True)
        self.log("at epoch", self.best_val_epoch, on_epoch=True)
        return val_AUC

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-5)

In [ ]:
max_epochs = 3 # number of epochs (whole iterations over the dataset) you would like to train for

class_len = 3
model = DeepDicomSort(num_classes=class_len) 
trainer = pl.Trainer(max_epochs=max_epochs)
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=test_loader)